In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sinkerDL = pd.read_csv('DraftLeagueSinker.csv')
sinkerMLB = pd.read_csv('MLBSinker.csv')

In [3]:
sinkerDL['league'] = 'Draft League'
sinkerMLB['league'] = 'MLB'

In [4]:
sinkerDL = sinkerDL.astype(str)
sinkerMLB = sinkerMLB.astype(str)

In [5]:
pitchersDL = list(set(sinkerDL['pitcher']))

In [6]:
slider = pd.concat([sinkerDL, sinkerMLB])

In [7]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [8]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [9]:
def get_index_from_pitcher(pitcher):
    return sinker[sinker.pitcher == pitcher].index[0]

In [10]:
def get_pitcher_from_index(index):
    return sinker[sinker.index == index].values[0][0]

In [11]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        sinker = pd.concat([sinkerDL, sinkerMLB])

        sinker["combined_features"] = sinker.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(sinker["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        sinker = sinker[(sinker['pitcher'] == pitcher_like) | (sinker['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass

In [12]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
0,"Wilhite, Thomas",No Comp,No Similarity
1,"Wojtarowicz, Andre","Hand, Brad",0.316228
2,"Wilson, Benjamin","Ureña, José",0.350823
3,"Oswald, Wesley","Szapucki, Thomas",0.33541
4,"Schomberg, Will","Gallen, Zac",0.316228
...,...,...,...
107,"Figueroa, Anthony","Muller, Kyle",0.4
108,"Gilchrist, Malcolm","Peterson, David",0.316228
109,"Wright, Kerry","Cederlind, Blake",0.435194
110,"Reeves, Cobe","Familia, Jeurys",0.381385


In [13]:
cosine_comp.to_csv('sinker_cosine_comp.csv', sep='\t')